<font color = green >

# Text classification: topic modeling 

</font>

<font color = green >

### Latent Dirichlet allocation (LDA)

</font>

Typically used to detect underlying topics in the text documents

**Input** : text documents and number of topics 
<br>
**Output**: Distribution of topics for each document (that allows to assign th one with highest probability) and word distribution for each topic 

**Assumptions**:
- Documents with similar topics use similar groups of words 
- Documents are probability distribution over latent topics 
- Topics are probability distribution over words


<font color = green >

#### Generative process
<br>
</font>

LDA considers the every document is created the following way:

1) Define number if words in the document
<br>
2) Chose the topic mixture over the fixed set of topics (e.g. 20% of topic A, 30% of topic A, and 50% of topic A)
<br>
3) Generate the words by:
<br>
   -pick the topic based on document's multinomial distribution 
<br>
   -pick the word based on topic's multinomial distribution 

<img src = "img/topics_modeling.png" height=500 width= 800 align="left">



<font color = green >

#### Recall
</font>


#### Binomial distribution

$$p(k/n)\quad =\quad C^{ k }_{ n }\cdot p^{ k }(1-p)^{ n-k }\quad =\quad \frac { n! }{ k!(n-k)! } p^{ k }(1-p)^{ n-k }$$

Example: Probability of 6 of 10 for fear coin: 
$$p(6,4)\quad =\quad C^{ 6 }_{ 10 }\cdot {0.5}^{ 6 }(0.5)^{ 4 }\quad = 210 \cdot 0.015625 \cdot 0.0625 = 0.205078125$$


#### Multinomial distribution

$$p(n_{ 1 }n_{ 2 }...n_{ k })\quad =\quad \frac { n! }{ n_{ 1 }!n_{ 2 }!...n_{ k }! } p^{ n_{ 1 } }_{ 1 }p^{ n_{ 2 } }_{ 2 }...p^{ n_{ k } }_{ k }$$

Example (three outcomes): <br>
n = 12 (12 games are played),<br>
n1 = 7 (number won by Player A),<br>
n2 = 2 (number won by Player B),<br>
n3 = 3 (the number drawn),<br>
p1 = 0.4 (probability Player A wins)<br>
p2 = 0.35(probability Player B wins)<br>
p3 = 0.25(probability of a draw)<br>
$$p(7,2,3)\quad =\quad \frac {12!}{ 7! \cdot 2! \cdot3 ! }  \cdot 0.4^{7} \cdot 0.35^{2} \cdot0.25^{3} = 0.0248$$




<font color = green >

#### Maximul Likelihood Estimation

</font>

<br>

**Recall** 
<br> Known are text documents and number $K$ of topics 

**Target**:
<br>Within all possible topics distribution for all documemnts and all possible words distribution for topics, shoose the one wich maximizes probability of all text documents.

**Approach** :
<br>
1) Randomly assign each word of each document to $K$ topics 
<br>
2) Iterate the following process till convergence (steady assignments of w to topics) 
<br>$\quad\quad$For each document $d$: 
<br>
    $\quad\quad\bullet$ Assume that all topic assignment except current one are correct     
    $\quad\quad\bullet$ For each word $w$ in $d$:           
    $\quad\quad\quad$ - For every topic $t$ compare the the score for hypothesis that w is in this topic $t$:
   <br>$\quad\quad\quad\quad\quad score (t) =  p(t | d) \cdot p (w |t),$
   <br>$\quad\quad\quad\quad p(t|d)$ is proportion of all words in d from t,
    <br>$\quad\quad\quad\quad p(w|t)$ is share of word w in topic t.  
    $\quad\quad\quad$ - Assign the word w to the topic with max score
    <br>$\quad\quad\bullet$ Iterate through all $w$ in $d$:           
$\quad\quad$Iterate through all $d$

Te results is matrix of distribution of words in topics  
Note: The computed topics are just words distribution, i.e. need to summarize them somehow. 


<font color = green >

## Gensim LDA 

</font>



In [1]:
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
from nltk.corpus import stopwords

<font color = green >

### Define the text documents 

</font>



In [2]:
doc_a = "Brocolli is good to eat. My brother likes to eat good brocolli, but not my mother."
doc_b = "My mother spends a lot of time driving my brother around to baseball practice."
doc_c = "Some health experts suggest that driving may cause increased tension and blood pressure."
doc_d = "I often feel pressure to perform well at school, but my mother never seems to drive my brother to do better."
doc_e = "Health professionals say that brocolli is good for your health."
doc_set = [doc_a, doc_b, doc_c, doc_d, doc_e]


<font color = green >

### Tokenize, clean, and stem

</font>



In [3]:
en_stop  = set(stopwords.words('english'))
p_stemmer = PorterStemmer()

def tokenize(doc_set):
    texts = []
    for doc in doc_set:
        # tokenize document string
        raw = doc.lower()
        tokens = word_tokenize(raw)

        # remove stop words from tokens
        tokens = [token for token in tokens if token not in en_stop]

        # stem tokens
        tokens = [p_stemmer.stem(token) for token in tokens]

        # add tokens to list
        texts.append(tokens)
    return texts

texts = tokenize(doc_set)
texts[0]

['brocolli',
 'good',
 'eat',
 '.',
 'brother',
 'like',
 'eat',
 'good',
 'brocolli',
 ',',
 'mother',
 '.']

<font color = green >

### Convert tokenized documents into a "id <-> term" dictionary

</font>



In [4]:
dictionary = corpora.Dictionary(texts) # this is alternative way - without using count vectorizer
print (type(dictionary), dictionary)
for k,w in dictionary.items():
    print (k,w)

<class 'gensim.corpora.dictionary.Dictionary'> Dictionary(34 unique tokens: [',', '.', 'brocolli', 'brother', 'eat']...)
0 ,
1 .
2 brocolli
3 brother
4 eat
5 good
6 like
7 mother
8 around
9 basebal
10 drive
11 lot
12 practic
13 spend
14 time
15 blood
16 caus
17 expert
18 health
19 increas
20 may
21 pressur
22 suggest
23 tension
24 better
25 feel
26 never
27 often
28 perform
29 school
30 seem
31 well
32 profession
33 say


<font color = green >

### Create gensim corpus

</font>



In [5]:
print ('\nconvert tokenized documents into a document-term matrix')
corpus = [dictionary.doc2bow(text) for text in texts] # id and count
for item in corpus:
    print (item)


convert tokenized documents into a document-term matrix
[(0, 1), (1, 2), (2, 2), (3, 1), (4, 2), (5, 2), (6, 1), (7, 1)]
[(1, 1), (3, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1)]
[(1, 1), (10, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1)]
[(0, 1), (1, 1), (3, 1), (7, 1), (10, 1), (21, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1)]
[(1, 1), (2, 1), (5, 1), (18, 2), (32, 1), (33, 1)]


<font color = green >

### Generate LDA model

</font>



In [6]:
ldamodel = gensim.models.ldamodel.LdaModel(
    corpus, num_topics=2, id2word=dictionary, passes=20, random_state= 0)


<font color = green >

### Review topics 

</font>



In [7]:
ldamodel.print_topics(num_topics=2,num_words=10)

[(0,
  '0.098*"." + 0.076*"brocolli" + 0.076*"good" + 0.055*"mother" + 0.055*"brother" + 0.054*"health" + 0.054*"eat" + 0.033*"," + 0.033*"like" + 0.033*"spend"'),
 (1,
  '0.060*"drive" + 0.059*"pressur" + 0.059*"." + 0.036*"," + 0.036*"never" + 0.036*"often" + 0.036*"increas" + 0.036*"perform" + 0.036*"seem" + 0.036*"well"')]

<font color = green >

### Classify the new text 

</font>



In [8]:
test_doc_list = ["Some experts suggest that car may cause increased blood pressure. professionals say that brocolli is good "]
test_texts = tokenize(test_doc_list)
test_corpus = [dictionary.doc2bow(text) for text in test_texts ]
test_doc_topics = ldamodel.get_document_topics(test_corpus)
print ('\nget topics:')
for el in test_doc_topics: # loop over all tests in provided list
    print(el)


get topics:
[(0, 0.37417653), (1, 0.62582344)]


<font color = green >

### Sample of topic modeling on large dataset

</font>



In [9]:
from sklearn.feature_extraction.text import CountVectorizer
import pickle

<font color = green >

#### Load "News" data 

</font>



In [10]:
import os
cwd= os.getcwd()
path = os.path.join(cwd,)
fn=  os.path.join(path , 'newsgroups')

with open(fn, 'rb') as f:
    newsgroup_data = pickle.load(f)

<font color = green >

#### Review data

</font>



In [11]:
print (type(newsgroup_data))
print ('len of documents = {:,}\n'.format(len(newsgroup_data)))

newsgroup_data[0]

<class 'list'>
len of documents = 2,000



"The best group to keep you informed is the Crohn's and Colitis Foundation\nof America.  I do not know if the UK has a similar organization.  The\naddress of\nthe CCFA is \n\nCCFA\n444 Park Avenue South\n11th Floor\nNew York, NY  10016-7374\nUSA\n\nThey have a lot of information available and have a number of newsletters.\n \nGood Luck."

<font color = green >

#### Define custom vectorizer

</font>



In [12]:
three_words_pattern = r"\b\w{3,}\b"
vectorizer = CountVectorizer(
    min_df=20, 
    stop_words='english',
    token_pattern=three_words_pattern) 
vectorizer.fit(newsgroup_data)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=20,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='\\b\\w{3,}\\b', tokenizer=None,
        vocabulary=None)

<font color = green >

#### Review feratures 

</font>



In [13]:
print ('len of features = {:,}\n'.format(len(vectorizer.get_feature_names())))
print (vectorizer.get_feature_names()[:40])


len of features = 902

['000', '100', '1990', '1992', '1993', '200', '2nd', '300', '400', '486', '500', '800', 'ability', 'able', 'accept', 'accepted', 'access', 'according', 'actual', 'actually', 'add', 'addition', 'additional', 'address', 'advance', 'advice', 'age', 'ago', 'agree', 'ahead', 'air', 'allow', 'alt', 'america', 'american', 'answer', 'answers', 'anybody', 'apparently', 'appears']


<font color = green >

#### Vectorize data set

</font>



In [14]:
newsgroup_data_vectorized= vectorizer.transform(newsgroup_data)
print (newsgroup_data_vectorized)

  (0, 23)	1
  (0, 33)	1
  (0, 58)	1
  (0, 76)	1
  (0, 326)	1
  (0, 335)	1
  (0, 386)	1
  (0, 409)	1
  (0, 451)	1
  (0, 456)	1
  (0, 515)	1
  (0, 529)	1
  (0, 545)	1
  (0, 727)	1
  (0, 843)	1
  (0, 900)	1
  (1, 33)	1
  (1, 34)	1
  (1, 84)	1
  (1, 184)	1
  (1, 201)	1
  (1, 214)	1
  (1, 231)	2
  (1, 241)	1
  (1, 324)	1
  :	:
  (1998, 622)	1
  (1998, 625)	3
  (1998, 688)	1
  (1998, 698)	2
  (1998, 726)	1
  (1998, 804)	1
  (1998, 805)	1
  (1998, 810)	10
  (1998, 813)	2
  (1998, 814)	1
  (1998, 816)	1
  (1998, 818)	1
  (1998, 844)	1
  (1998, 882)	2
  (1998, 899)	1
  (1999, 171)	1
  (1999, 194)	1
  (1999, 205)	1
  (1999, 213)	1
  (1999, 276)	2
  (1999, 308)	1
  (1999, 344)	1
  (1999, 669)	1
  (1999, 832)	1
  (1999, 874)	1


<font color = green >

#### Create gensim corpus

</font>



In [15]:
corpus = gensim.matutils.Sparse2Corpus(newsgroup_data_vectorized, documents_columns=False)
# comparing to using corpora.Dictionary:
# corpus = [dictionary.doc2bow(text) for text in texts] 
[item for item in corpus][:5]


[[(23, 1),
  (33, 1),
  (58, 1),
  (76, 1),
  (326, 1),
  (335, 1),
  (386, 1),
  (409, 1),
  (451, 1),
  (456, 1),
  (515, 1),
  (529, 1),
  (545, 1),
  (727, 1),
  (843, 1),
  (900, 1)],
 [(33, 1),
  (34, 1),
  (84, 1),
  (184, 1),
  (201, 1),
  (214, 1),
  (231, 2),
  (241, 1),
  (324, 1),
  (332, 1),
  (359, 1),
  (363, 1),
  (365, 1),
  (409, 1),
  (430, 3),
  (451, 1),
  (475, 1),
  (492, 2),
  (525, 2),
  (605, 1),
  (633, 2),
  (642, 1),
  (674, 1),
  (688, 1),
  (709, 1),
  (750, 1),
  (777, 1),
  (823, 1),
  (838, 1),
  (874, 1),
  (896, 1)],
 [(25, 1),
  (26, 1),
  (63, 1),
  (120, 1),
  (231, 1),
  (297, 1),
  (326, 1),
  (344, 1),
  (373, 1),
  (423, 1),
  (442, 1),
  (444, 1),
  (448, 2),
  (465, 1),
  (572, 1),
  (653, 1),
  (659, 1),
  (714, 1),
  (777, 1),
  (779, 1),
  (781, 1),
  (818, 1),
  (836, 1),
  (855, 1),
  (890, 1),
  (898, 1)],
 [(4, 1),
  (17, 2),
  (18, 1),
  (22, 1),
  (42, 1),
  (48, 2),
  (68, 1),
  (78, 1),
  (86, 1),
  (94, 1),
  (117, 1),
  (119, 1)

<font color = green >

#### Create id2word dictionary

</font>



In [16]:
id_map = dict((v, k) for k, v in vectorizer.vocabulary_.items()) 

<font color = green >

#### Generate LDA model

</font>



In [17]:
ldamodel = gensim.models.ldamodel.LdaModel (corpus, num_topics=6, id2word=id_map, passes=25, random_state=34)
# Comparing to corpora.Dictionary
# ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=2, id2word=dictionary, passes=20, random_state= 0)


<font color = green >

#### Review topics

</font>



In [18]:
ldamodel.print_topics(num_topics=6,num_words=10)

[(0,
  '0.025*"edu" + 0.019*"com" + 0.018*"use" + 0.018*"thanks" + 0.016*"does" + 0.015*"know" + 0.011*"mail" + 0.010*"apple" + 0.009*"help" + 0.008*"want"'),
 (1,
  '0.061*"drive" + 0.039*"disk" + 0.030*"scsi" + 0.027*"drives" + 0.027*"hard" + 0.025*"controller" + 0.021*"card" + 0.018*"rom" + 0.016*"cable" + 0.016*"floppy"'),
 (2,
  '0.024*"people" + 0.022*"god" + 0.012*"atheism" + 0.012*"think" + 0.012*"believe" + 0.012*"don" + 0.010*"does" + 0.010*"just" + 0.009*"argument" + 0.009*"say"'),
 (3,
  '0.023*"game" + 0.021*"year" + 0.020*"team" + 0.013*"games" + 0.013*"play" + 0.011*"good" + 0.011*"don" + 0.010*"think" + 0.010*"season" + 0.010*"players"'),
 (4,
  '0.035*"space" + 0.019*"nasa" + 0.018*"data" + 0.013*"information" + 0.013*"available" + 0.013*"center" + 0.011*"ground" + 0.010*"research" + 0.010*"000" + 0.010*"new"'),
 (5,
  '0.017*"just" + 0.017*"like" + 0.016*"don" + 0.012*"car" + 0.012*"time" + 0.011*"think" + 0.011*"good" + 0.010*"know" + 0.008*"way" + 0.008*"people"')]

<font color = green >

#### Name topics

</font>



In [19]:
topics_names= ['Education', 'Computers & IT', 'Religion', 'Sports', 'Science','Society & Lifestyle']

<font color = green >

#### Classify the new text 

</font>



In [20]:
import numpy as np

In [21]:
new_doc = ["\n\nIt's my understanding that the freezing will start to occur because \
of the\ngrowing distance of Pluto and Charon from the Sun, due to it's\nelliptical orbit. \
It is not due to shadowing effects. \n\n\nPluto can shadow Charon, and vice-versa.\n\nGeorge \
Krumins\n-- "] 


In [22]:
doc_vectorized= vectorizer.transform(new_doc) # input param is list
new_doc_corpus = gensim.matutils.Sparse2Corpus(doc_vectorized, documents_columns=False)
doc_topics = ldamodel.get_document_topics(new_doc_corpus)
list(doc_topics)

[[(0, 0.033417594),
  (1, 0.033340912),
  (2, 0.03351659),
  (3, 0.033779576),
  (4, 0.8323054),
  (5, 0.03363992)]]

In [23]:
def elicit_topic_name(doc_topics):    
    return topics_names[np.squeeze(np.array(doc_topics))[:,1].argmax()]
elicit_topic_name(doc_topics)

'Science'

<font color = green >

## Home Task 

</font>


<font color = green >

### Topic Modeling 

</font>

[voted-kaggle-dataset](https://www.kaggle.com/canggih/voted-kaggle-dataset/version/2#voted-kaggle-dataset.csv)

In [84]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
from nltk.corpus import stopwords
import pandas as pd
import gensim
import re

fn=  os.path.join(path , 'voted-kaggle-dataset.csv')
df = pd.read_csv(fn)

In [85]:
df.head(50)

,Title,Subtitle,Owner,Votes,Versions,Tags,Data Type,Size,License,Views,Download,Kernels,Topics,URL,Description
0,Credit Card Fraud Detection,Anonymized credit card transactions labeled as...,Machine Learning Group - ULB,1241,"Version 2,2016-11-05|Version 1,2016-11-03",crime\nfinance,CSV,144 MB,ODbL,"442,136 views","53,128 downloads","1,782 kernels",26 topics,https://www.kaggle.com/mlg-ulb/creditcardfraud,The datasets contains transactions made by cre...
1,European Soccer Database,"25k+ matches, players & teams attributes for E...",Hugo Mathien,1046,"Version 10,2016-10-24|Version 9,2016-10-24|Ver...",association football\neurope,SQLite,299 MB,ODbL,"396,214 views","46,367 downloads","1,459 kernels",75 topics,https://www.kaggle.com/hugomathien/soccer,The ultimate Soccer database for data analysis...
2,TMDB 5000 Movie Dataset,"Metadata on ~5,000 movies from TMDb",The Movie Database (TMDb),1024,"Version 2,2017-09-28",film,CSV,44 MB,Other,"446,255 views","62,002 downloads","1,394 kernels",46 topics,https://www.kaggle.com/tmdb/tmdb-movie-metadata,Background\nWhat can we say about the success ...
3,Global Terrorism Database,"More than 170,000 terrorist attacks worldwide,...",START Consortium,789,"Version 2,2017-07-19|Version 1,2016-12-08",crime\nterrorism\ninternational relations,CSV,144 MB,Other,"187,877 views","26,309 downloads",608 kernels,11 topics,https://www.kaggle.com/START-UMD/gtd,"Context\nInformation on more than 170,000 Terr..."
4,Bitcoin Historical Data,Bitcoin data at 1-min intervals from select ex...,Zielak,618,"Version 11,2018-01-11|Version 10,2017-11-17|Ve...",history\nfinance,CSV,119 MB,CC4,"146,734 views","16,868 downloads",68 kernels,13 topics,https://www.kaggle.com/mczielinski/bitcoin-his...,Context\nBitcoin is the longest running and mo...
5,"Kaggle ML and Data Science Survey, 2017",A big picture view of the state of data scienc...,Kaggle,574,"Version 4,2017-10-28|Version 3,2017-10-03|Vers...",employment\nsociology\nartificial intelligence,CSV,28 MB,ODbL,"95,587 views","9,390 downloads",244 kernels,10 topics,https://www.kaggle.com/kaggle/kaggle-survey-2017,"Context\nFor the first time, Kaggle conducted ..."
6,Iris Species,Classify iris plants into three species in thi...,UCI Machine Learning,512,"Version 2,2016-09-27|Version 1,2016-01-12",botany,SQLite,15 KB,CC0,"162,706 views","24,361 downloads","3,394 kernels",14 topics,https://www.kaggle.com/uciml/iris,The Iris dataset was used in R.A. Fisher's cla...
7,World Development Indicators,Explore country development indicators from ar...,World Bank,468,"Version 2,2017-05-02|Version 1,2016-01-28",economics\ninternational relations,CSV,2 GB,Other,"134,038 views","20,364 downloads",389 kernels,5 topics,https://www.kaggle.com/worldbank/world-develop...,The World Development Indicators from the Worl...
8,Daily News for Stock Market Prediction,Using 8 years daily news headlines to predict ...,Aaron7sun,438,"Version 1,2016-08-25",news agencies\nfinance,CSV,14 MB,CC4,"95,770 views","10,820 downloads",293 kernels,8 topics,https://www.kaggle.com/aaron7sun/stocknews,"Actually, I prepare this dataset for students ..."
9,Pokemon with stats,721 Pokemon with stats and types,Alberto Barradas,428,"Version 2,2016-08-29|Version 1,2016-08-23",popular culture\ngames and toys\nvideo games,CSV,43 KB,CC0,"133,256 views","16,610 downloads",706 kernels,13 topics,https://www.kaggle.com/abcsds/pokemon,"This data set includes 721 Pokemon, including ..."


In [86]:
print ('len of texts= {:,}'.format(len(df)))
index = 10 
df.loc[index, 'Description']

len of texts= 2,150


'These files contain complete loan data for all loans issued through the 2007-2015, including the current loan status (Current, Late, Fully Paid, etc.) and latest payment information. The file containing loan data through the "present" contains complete loan data for all loans issued through the previous completed calendar quarter. Additional features include credit scores, number of finance inquiries, address including zip codes, and state, and collections among others. The file is a matrix of about 890 thousand observations and 75 variables. A data dictionary is provided in a separate file. k'

In [87]:
df_data_set = df["Description"]
# df["Description"].apply(pd.value_counts) //Note: not working for non numbers type 

In [90]:
en_stop  = set(stopwords.words('english'))
p_stemmer = PorterStemmer()

def tokenize(df_data_set):
    texts = []
    for doc in df_data_set:
        # tokenize document string
        doc = str(doc)
        raw = doc.lower()
        tokens = word_tokenize(raw)

        # remove stop words from tokens
        tokens = [token for token in tokens if token not in en_stop]
        
        #remove words that length are less than 3 chars

        tokens =  [token for token in tokens if len(token) >3]
                
        # stem tokens
        tokens = [p_stemmer.stem(token) for token in tokens]

        # add tokens to list
        texts.append(tokens)
      
    return texts

texts = tokenize(df_data_set)
texts[0]

['dataset',
 'contain',
 'transact',
 'made',
 'credit',
 'card',
 'septemb',
 '2013',
 'european',
 'cardhold',
 'dataset',
 'present',
 'transact',
 'occur',
 'day',
 'fraud',
 '284,807',
 'transact',
 'dataset',
 'highli',
 'unbalanc',
 'posit',
 'class',
 'fraud',
 'account',
 '0.172',
 'transact',
 'contain',
 'numer',
 'input',
 'variabl',
 'result',
 'transform',
 'unfortun',
 'confidenti',
 'issu',
 'provid',
 'origin',
 'featur',
 'background',
 'inform',
 'data',
 'featur',
 'princip',
 'compon',
 'obtain',
 'featur',
 'transform',
 "'time",
 "'amount",
 'featur',
 "'time",
 'contain',
 'second',
 'elaps',
 'transact',
 'first',
 'transact',
 'dataset',
 'featur',
 "'amount",
 'transact',
 'amount',
 'featur',
 'use',
 'example-depend',
 'cost-senst',
 'learn',
 'featur',
 "'class",
 'respons',
 'variabl',
 'take',
 'valu',
 'case',
 'fraud',
 'otherwis',
 'given',
 'class',
 'imbal',
 'ratio',
 'recommend',
 'measur',
 'accuraci',
 'use',
 'area',
 'precision-recal',
 'curv'

In [56]:
# moby_frequencies = FreqDist(nltk.word_tokenize(texts))


In [91]:
dictionary = corpora.Dictionary(texts) # this is alternative way - without using count vectorizer
print (type(dictionary), dictionary)
for k,w in dictionary.items():
    print (k,w)

<class 'gensim.corpora.dictionary.Dictionary'> Dictionary(32459 unique tokens: ["'amount", "'class", "'time", '//mlg.ulb.ac.b', '//mlg.ulb.ac.be/artml']...)
0 'amount
1 'class
2 'time
3 //mlg.ulb.ac.b
4 //mlg.ulb.ac.be/artml
5 //mlg.ulb.ac.be/bruf
6 0.172
7 2013
8 2015
9 284,807
10 account
11 accuraci
12 amount
13 analys
14 andrea
15 area
16 auprc
17 avail
18 background
19 bontempi
20 bruxel
21 caelen
22 calibr
23 card
24 cardhold
25 case
26 cidm
27 cite
28 class
29 classif
30 collabor
31 collect
32 compon
33 comput
34 confidenti
35 confus
36 contain
37 cost-senst
38 credit
39 current
40 curv
41 data
42 dataset
43 day
44 detail
45 detect
46 elaps
47 european
48 example-depend
49 featur
50 first
51 fraud
52 gianluca
53 given
54 group
55 highli
56 http
57 ieee
58 imbal
59 inform
60 input
61 intellig
62 issu
63 johnson
64 learn
65 libr
66 machin
67 made
68 matrix
69 meaning
70 measur
71 mine
72 numer
73 obtain
74 occur
75 olivi
76 origin
77 otherwis
78 past
79 pleas
80 posit
81 pozzolo
82

764 unintuit
765 unix
766 variou
767 viewer
768 volum
769 wait
770 weight
771 -freeformresponses.csv
772 1000
773 16,000
774 16,716
775 16.4
776 25th
777 allow
778 attempt
779 august
780 break
781 calcul
782 channel
783 choic
784 column-wis
785 compens
786 comprehens
787 conduct
788 convers
789 conversionrates.csv
790 correspond
791 cut
792 day-to-day
793 decod
794 describ
795 edg
796 email
797 employ
798 establish
799 exclud
800 flag
801 folk
802 form
803 free
804 freeform
805 freeformresponses.csv
806 happen
807 ident
808 industry-wid
809 initi
810 interact
811 kind
812 less
813 live
814 median
815 month
816 multipl
817 multiplechoiceresponses.csv
818 non-random
819 novemb
820 open-end
821 option
822 packag
823 primarili
824 proceed
825 protect
826 pud
827 quantmod
828 rank
829 read
830 receiv
831 regard
832 relev
833 respond
834 respondenttypereadme.txt
835 salari
836 schema.csv
837 scientist
838 segment
839 shown
840 singl
841 spam
842 survey
843 survey-tak
844 territori
845 total


1513 today
1514 un-reli
1515 under-explain
1516 unexplain
1517 unpleas
1518 utopia
1519 well-b
1520 whole
1521 width
1522 //github.com/zalandoresearch/fashion-mnist
1523 //pjreddie.com/projects/mnist-in-csv/
1524 //tech.zalando.com
1525 10,000
1526 255.
1527 28x28
1528 60,000
1529 ai/ml/data
1530 and/or
1531 ankl
1532 aris
1533 ascii-diagram
1534 assign
1535 boot
1536 charg
1537 claim
1538 cloth
1539 coat
1540 commun
1541 connect
1542 convert
1543 copi
1544 copyright
1545 dark
1546 darker
1547 decompos
1548 direct
1549 dress
1550 drop-in
1551 fail
1552 fashion-mnist
1553 fit
1554 furnish
1555 grayscal
1556 handwritten
1557 height
1558 herebi
1559 holder
1560 images—consist
1561 impli
1562 integ
1563 intend
1564 left
1565 licens
1566 light
1567 merchant
1568 merg
1569 mnist
1570 noninfring
1571 permit
1572 pixel
1573 pixel-valu
1574 pixel31
1575 pullov
1576 remain
1577 rest
1578 restrict
1579 sandal
1580 seek
1581 sell
1582 shirt
1583 size
1584 sneaker
1585 sublicens
1586 substanti
1587

2263 librari
2264 neutral
2265 reformat
2266 rude
2267 tweet
2268 u.s.
2269 0hz-280hz
2270 140hz
2271 3,168
2272 absolut
2273 accumul
2274 acoust
2275 adjac
2276 appar
2277 baselin
2278 carnegi
2279 cart
2280 centroid
2281 cmu_arct
2282 corpu
2283 data-set
2284 dfrang
2285 diagram
2286 domin
2287 energi
2288 entropi
2289 falsetto
2290 festvox
2291 flat
2292 forest
2293 harvard-haskin
2294 incorrectli
2295 interquantil
2296 kurt
2297 kurtosi
2298 logist
2299 maxdom
2300 maxfun
2301 mcgill
2302 meandom
2303 meanfreq
2304 meanfun
2305 mellon
2306 mindom
2307 minfun
2308 mode
2309 modindx
2310 peakf
2311 pre-process
2312 quantil
2313 regular
2314 regularly-tim
2315 reson
2316 seewav
2317 signal
2318 skew
2319 sp.ent
2320 specprop
2321 spectral
2322 speech
2323 telecommun
2324 threshold
2325 tuner
2326 vocal
2327 voic
2328 voxforg
2329 xgboost
2330 0.2882
2331 0.3843
2332 0.3881
2333 0.5438
2334 0.7813
2335 03/01/2018
2336 0x375923bf82f0b728d23a5704261a6e16341fd860
2337 1,382
2338 105.21
23

3013 suffici
3014 whizzo
3015 yeah
3016 ..lub……….dub………..………
3017 //www.peterjbentley.com/heartchallenge/index.html
3018 32gb
3019 abnorm
3020 adult
3021 aistat
3022 anywher
3023 artifact
3024 asterisk*
3025 audio
3026 automat
3027 beat
3028 bentley
3029 bodi
3030 breath
3031 brush
3032 calm
3033 chsc2011
3034 citat
3035 clear
3036 clinic
3037 clip
3038 coimbra
3039 consider
3040 cycl
3041 devic
3042 digiscop
3043 discern
3044 diseas
3045 distinguish
3046 distort
3047 domain-specif
3048 dub-dub
3049 dub.dub………………….lub.……….dub.dub………………….lub……..…….dub
3050 dub…**….lub
3051 dub…*….lub
3052 dub……
3053 dub……………
3054 earlier
3055 eart
3056 echo
3057 enabl
3058 esult
3059 extrasystol
3060 fluid
3061 fname
3062 fourier
3063 fragment
3064 gold-standard
3065 halleng
3066 heartbeat
3067 hospit
3068 howpublish
3069 ignor
3070 illustr
3071 ipad
3072 istethoscop
3073 lassifi
3074 low-pass
3075 lub-lub
3076 lub..*..dub
3077 lub……….dub…
3078 lub……….dub……………
3079 lub……….dub…………….lub.lub……..…….dub……
30

...

29004 sohinibhattacharya
29005 solanki
29006 soldo
29007 soler-compani
29008 somasundaram
29009 somat
29010 somavarapu
29011 somesh
29012 sommenoob
29013 somnath
29014 sonali
29015 sonamsrivastava
29016 sonavan
29017 sonawan
29018 songsong
29019 sonneland
29020 soojung
29021 soorajm
29022 soroosh
29023 sotopia
29024 souci
29025 soufian
29026 soufianeorama
29027 souhaiel
29028 souhail
29029 soukaina
29030 souliman
29031 souman
29032 soumitra
29033 sourabhmitt
29034 sourav
29035 souravmaharana
29036 sousa
29037 sovboc2018
29038 sowhit
29039 sowmiya
29040 soywu
29041 spadafora
29042 spadim
29043 spencer
29044 sprabakar
29045 sreeram
29046 sreyansh
29047 sridhar
29048 sridharan
29049 srigiriraju
29050 srihari
29051 sriharirao
29052 srikanth
29053 srilakshmi
29054 srilakshminandamuri
29055 srilbg
29056 srinath
29057 srinivasan
29058 srinivasrao
29059 srinivinnakota
29060 srivastava
29061 srk16113
29062 ssvitian
29063 stamat
29064 stanciu
29065 stanford-taylor
29066 starconf
29067 stawari
29

29754 is_respons
29755 cant
29756 ml/dp
29757 motherboard
29758 science~
29759 solder
29760 //stats.espncricinfo.com/ci/engine/stats/index.html
29761 all-round
29762 statsguru
29763 1800-2020.
29764 11/14/2017
29765 sci-kit
29766 //www.epa.gov/tsca-inventory/how-access-tsca-inventori
29767 1975.
29768 62,000
29769 accno
29770 casregno
29771 number/form
29772 pmnacc_062017.csv
29773 pmnno
29774 tsca
29775 tscainv_062017.csv
29776 uvcb
29777 crypto-curr
29778 mosquito-blueprint
29779 vericoin
29780 brainchild
29781 elsewher
29782 erect
29783 ewart
29784 plaqu
29785 plauq
29786 //www.dds.ca.gov
29787 developmentally-dis
29788 mickel
29789 non-hispan
29790 paradox
29791 sacremento
29792 www.amstat.org/publications/jse/v22n1/mickel.pdf
29793 applicatin
29794 feauter
29795 seper
29796 10,502
29797 all_annotated.tsv
29798 blodgett
29799 code-switch
29800 geotag
29801 johnni
29802 o'connor
29803 tian-zheng
29804 978-9077381-39-7
29805 gutenbergr
29806 janeaustenr
29807 page-by-pag
29808 word-c

30503 multi-bleu.perl
30504 newstest
30505 newstest-2015-100sents.en-ru.google.ru
30506 newstest-2015-100sents.en-ru.ref.en
30507 newstest-2015-100sents.en-ru.ref.ru
30508 newstest-2015-100sents.en-ru.src.en
30509 newstest-2015.enru.ref.ru.sgm
30510 newstest-2015.enru.src.en.sgm
30511 newstest2015-100sents-enru-google.ru.sgm
30512 newstest2015-100sents-enru-ref.ru.sgm
30513 newstest2015-100sents-enru-src.en.sgm
30514 newstest2015-enru-ref.ru.sgm
30515 newstest2015-enru-src.en.sgm
30516 nltk.translat
30517 plaintext
30518 ratio=1.074
30519 ref.ru
30520 ref_len=1852
30521 src.en
30522 testb
30523 tokenizer.perl
30524 wmt15
30525 wmt15_eval
30526 wmt16
30527 wrap-xml.perl
30528 ~/mosesdecoder/scripts/ems/support/wrap-xml.perl
30529 ~/mosesdecoder/scripts/generic/mteval-v13a.pl
30530 ~/mosesdecoder/scripts/generic/multi-bleu.perl
30531 ~/mosesdecoder/scripts/tokenizer/tokenizer.perl
30532 7.0.0
30533 close_punctuation.txt
30534 currency_symbol.txt
30535 isalnum.txt
30536 isalpha.txt
30537 

31253 hausa
31254 isizulu
31255 iso_codes.csv
31256 kiswahili
31257 lugbarati
31258 open-licens
31259 sesotho
31260 somali
31261 stoplist
31262 storybook
31263 yoruba
31264 dlabal
31265 garageband
31266 midi
31267 wedeen
31268 bermuda.csv
31269 canada.csv
31270 curaçao
31271 curaçao.csv
31272 jamaica.csv
31273 mexico.csv
31274 argentina.csv
31275 brazil.csv
31276 chile
31277 chile.csv
31278 columbia.csv
31279 uraguay
31280 uraguay.csv
31281 continuo
31282 mush
31283 unhappi
31284 abolish
31285 admidst
31286 circumscrib
31287 confront
31288 darwinist
31289 enlist
31290 ex-slav
31291 memoir
31292 modernist
31293 oppress
31294 ravag
31295 re-creat
31296 segreg
31297 sharecropp
31298 tumultu
31299 turmoil
31300 //catalog.archives.gov/id/2240988
31301 2240988.
31302 burial_statu
31303 ethnicity_1
31304 ethnicity_2
31305 fatality_d
31306 fatality_year
31307 home_c
31308 home_counti
31309 home_st
31310 hostility_condit
31311 pay_grad
31312 service_cod
31313 service_typ
31314 //twitter.com/lin

32003 fortnit
32004 projectil
32005 reload
32006 soumydev
32007 underutil
32008 //www.usnews.com
32009 acceptance-r
32010 act-avg
32011 adelphi
32012 akron
32013 albani
32014 alliant
32015 argosi
32016 ashland
32017 auburn
32018 augusta
32019 azusa
32020 baptist
32021 baylor
32022 benedictin
32023 biola
32024 bloomington
32025 bois
32026 brandei
32027 brigham
32028 brunswick
32029 canyon
32030 capella
32031 carbondal
32032 charlott
32033 cincinnati
32034 clarkson
32035 clemson
32036 cumberland
32037 dame
32038 dartmouth
32039 depaul
32040 dominion
32041 drexel
32042 duke
32043 duquesn
32044 edgewood
32045 emori
32046 fairbank
32047 fordham
32048 fresno
32049 fullerton
32050 gardner-webb
32051 greensboro
32052 hartford
32053 hofstra
32054 huntsvil
32055 immaculata
32056 indianapoli
32057 kennesaw
32058 kingsvil
32059 lafayett
32060 lamar
32061 lehigh
32062 lesley
32063 lindenwood
32064 lipscomb
32065 lowel
32066 loyola
32067 manoa
32068 marquett
32069 maryvil
32070 merc
32071 mercer
320

In [92]:
print ('\nconvert tokenized documents into a document-term matrix')
corpus = [dictionary.doc2bow(text) for text in texts] # id and count
for item in corpus:
    print (item)


convert tokenized documents into a document-term matrix
[(0, 2), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 2), (29, 2), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 3), (37, 1), (38, 1), (39, 1), (40, 1), (41, 3), (42, 5), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 7), (50, 1), (51, 4), (52, 1), (53, 1), (54, 1), (55, 1), (56, 3), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 2), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 7), (101, 2), (102, 3), (103, 1), (104, 1), (1

[(36, 1), (41, 5), (42, 3), (49, 2), (53, 1), (67, 1), (70, 1), (76, 3), (80, 1), (83, 1), (106, 1), (107, 3), (124, 1), (125, 2), (140, 1), (143, 6), (150, 2), (158, 1), (170, 1), (173, 1), (176, 1), (181, 1), (190, 3), (240, 1), (251, 3), (256, 3), (276, 1), (282, 1), (306, 5), (329, 1), (347, 1), (367, 4), (400, 1), (410, 1), (412, 1), (421, 2), (423, 1), (430, 8), (441, 1), (475, 1), (539, 2), (596, 1), (601, 1), (633, 1), (634, 1), (639, 2), (650, 1), (722, 1), (725, 1), (794, 1), (815, 1), (872, 1), (887, 1), (891, 1), (896, 1), (934, 1), (936, 1), (964, 1), (982, 5), (990, 1), (993, 1), (1040, 1), (1052, 1), (1053, 2), (1059, 1), (1069, 2), (1100, 3), (1276, 1), (1306, 1), (1400, 1), (1402, 1), (1444, 1), (1492, 1), (1496, 1), (1541, 1), (1544, 1), (1566, 5), (1598, 2), (1657, 1), (1663, 1), (1706, 1), (1777, 1), (1789, 1), (1828, 2), (1847, 1), (2044, 1), (2114, 2), (2185, 1), (2285, 1), (2313, 1), (2453, 1), (2703, 7), (2771, 4), (2916, 1), (2930, 1), (3029, 1), (3035, 1), (32

[(36, 1), (41, 3), (42, 1), (54, 1), (73, 1), (165, 1), (170, 1), (185, 1), (196, 1), (224, 1), (232, 1), (239, 1), (329, 1), (399, 1), (425, 1), (431, 2), (561, 1), (605, 1), (624, 1), (741, 1), (829, 1), (1364, 1), (1369, 1), (1540, 2), (1601, 1), (1668, 2), (1680, 1), (2884, 1), (3447, 5), (3465, 1), (3776, 1), (4313, 5), (4532, 1), (7788, 2), (9884, 4), (9893, 2), (11676, 1), (20066, 1)]
[(10, 1), (31, 1), (32, 2), (36, 2), (41, 6), (42, 3), (44, 2), (54, 1), (59, 2), (87, 1), (106, 1), (108, 2), (133, 1), (143, 2), (151, 1), (182, 1), (194, 1), (201, 1), (202, 2), (203, 1), (211, 3), (262, 3), (271, 1), (276, 1), (302, 1), (338, 2), (376, 3), (390, 2), (432, 1), (464, 1), (472, 1), (501, 2), (502, 1), (513, 1), (534, 1), (535, 1), (648, 4), (665, 1), (669, 1), (741, 1), (757, 1), (787, 1), (842, 6), (845, 2), (862, 3), (870, 1), (949, 3), (976, 1), (982, 1), (1107, 1), (1236, 1), (1350, 3), (1355, 1), (1484, 1), (1494, 1), (1565, 1), (1629, 1), (1750, 1), (1796, 1), (2016, 1), (22

[(42, 1), (896, 1)]
[(12, 1), (41, 5), (59, 3), (72, 1), (87, 4), (93, 3), (100, 4), (106, 1), (108, 1), (125, 1), (126, 2), (176, 1), (196, 1), (202, 1), (234, 1), (276, 1), (360, 1), (363, 1), (453, 1), (501, 1), (502, 1), (553, 1), (722, 1), (757, 1), (833, 1), (887, 1), (893, 2), (934, 1), (1107, 1), (1494, 1), (1540, 1), (1791, 4), (2703, 2), (2902, 1), (3271, 1), (3355, 1), (3407, 1), (3704, 1), (5187, 1), (32152, 1), (32153, 1)]
[(36, 1), (56, 1), (59, 3), (64, 1), (106, 1), (118, 1), (146, 2), (158, 1), (203, 1), (276, 1), (360, 1), (501, 1), (502, 1), (509, 1), (543, 1), (660, 1), (1045, 5), (2045, 1), (3407, 1), (3452, 2), (3465, 1), (3697, 1), (3834, 1), (4342, 6), (4670, 1), (6193, 5), (8118, 1), (8759, 1), (8765, 1), (9006, 1), (14793, 1), (18111, 1), (18116, 1), (18119, 1), (18120, 1), (18122, 1), (21694, 2), (32154, 1), (32155, 1), (32156, 1)]
[(41, 2), (42, 1), (67, 1), (232, 1), (395, 1), (501, 1), (502, 1), (523, 1), (742, 1), (990, 1), (1052, 1), (1534, 1), (2853, 1)

In [ ]:
ldamodel = gensim.models.ldamodel.LdaModel(
    corpus, num_topics=3, id2word=dictionary, passes=20, random_state= 0)

In [ ]:
ldamodel.print_topics(num_topics=5,num_words=15)

<font color = green >

## Learn more
</font>

Latent Dirichlet allocation
<br>
https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation


<font color = green >

## Next lesson: Clustering 
</font>

